# Notebook with test code

---

## Imports

### Standard library imports

In [1]:
import yaml

import sys

from datetime import date

import os

### Third party libraries

In [2]:
import boto3

import pandas as pd

### Local application imports

In [3]:
sys.path.append("../")

In [4]:
%load_ext autoreload
%autoreload 2

from src.pipeline.ingesta_almacenamiento import (
    
    ## Parameters
    bucket_name,
    token,
    today_info,
    hist_ingest_path,
    hist_dat_prefix,
    cont_ingest_path,
    cont_dat_prefix,
    
    ## Functions
    get_client,
    ingesta_inicial,
    ingesta_consecutiva,
    get_s3_resource,
    guardar_ingesta
    
)

ImportError: cannot import name 'token' from 'src.pipeline.ingesta_almacenamiento' (../src/pipeline/ingesta_almacenamiento.py)

---

## AWS base examples

### S3

#### Initial configuration

In [ ]:
ses = boto3.session.Session(profile_name="robper_dpa")

In [ ]:
s3 = ses.resource("s3")

In [ ]:
[buck.name for buck in s3.buckets.all()]

In [ ]:
# este archivo yaml vive en tu conf/local/credentials.yaml
# este va en src/utils general.py
def read_yaml_file(yaml_file):
    """ load yaml cofigurations """

    config = None
    try: 
        with open (yaml_file, 'r') as f:
            config = yaml.safe_load(f)
    except:
        raise FileNotFoundError('Couldnt load the file')
    
    return config

In [ ]:
# este va en src/utils general.py
def get_s3_credentials(credentials_file):
    credentials = read_yaml_file(credentials_file)
    s3_creds = credentials['s3']

    return s3_creds

In [ ]:
s3_creds = get_s3_credentials("../conf/local/credentials.yaml")

session = boto3.Session(
    aws_access_key_id=s3_creds['aws_access_key_id'],
    aws_secret_access_key=s3_creds['aws_secret_access_key']
)
s3 = session.client('s3')

#### Connecting to existing bucket and viewing contents

In [ ]:
bucket = "dpa-robtest"
key ="rob_test_dir_s3"

In [ ]:
objects = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents']
objects

#### Creating bucket

In [ ]:
bucket_name = "comdline-test-bucket-rob"

s3.create_bucket(
    Bucket=bucket_name,
    CreateBucketConfiguration={'LocationConstraint': 'us-west-2'},
    ACL="private"
)

#### Uploading file to bucket

In [ ]:
file_to_upload = "../../admin/test_file3_for_s3.txt"
file_name = file_to_upload.split(sep="/")[-1]

In [ ]:
# s3.upload_file(file_to_upload, bucket_name, file_name)

s3.meta.client.upload_file(file_to_upload, bucket_name, file_name)

In [ ]:
[obj for obj in s3.Bucket(bucket_name).objects.all()]

#### Downloading files from bucket

In [ ]:
path_to_download = "/Users/rp_mbp/Documents/ReposRob_RobPer/ITAMmcd/semestre_2/Arquitectura_Prod_Dat/Venv_ArqPD/repos/dpa_2021/_robdir/" + file_name

s3.meta.client.download_file(bucket_name, file_name, path_to_download)

#### Deleting bucket files

In [ ]:
s3.Object(bucket_name, file_name).delete()

#### Deleting bucket

---

## Downloading and storing Chicago data

#### Getting s3 resource

#### Uploading info

In [ ]:
# guardar_ingesta(bucket_name, hist_ingest_path)
guardar_ingesta(bucket_name, cont_ingest_path)

#### Checking aws s3 contents

In [ ]:
key_hist = "ingestion/initial"

In [ ]:
objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=key_hist)['Contents']
objects

---